python code to post-process mdEditorRead csv to template format for import into MBM national asset catalog

In [ ]:
import csv
import pandas as pd
import chardet
import re

inputCSV = 'C:\\Users\\tpatterson\\OneDrive - DOI\\Documents\\DM_Metadatafiles\\CatalogCSV\\catalogCSVmbm20240606.csv'
outCSV = 'C:\\Users\\tpatterson\\OneDrive - DOI\\Documents\\DM_Metadatafiles\\CatalogCSV\\catalogAssetsmbm20240606.csv'


#read csv into panda data frome
with open(inputCSV, 'rb') as f:
    result = chardet.detect(f.read())
df = pd.read_csv(inputCSV, encoding=result['encoding'])

#create df to edit
df2=df

#dd region column and code
reg = 7
df2['MBPRegion'] = 7
#add storage/Maintenance contracted? response
df2['ContractStorage'] = 'NO'

#pull AK RDR id from ids & write program name to new columns
pattern = 'AlaskaRegionalDataRepository'
for i, r in df2.iterrows():
    id = r['ids']
    id = id.replace(';', '')
    id = id.replace(':','')
    id = id.replace(' ','')
    index = id.find(pattern)       
    df2.loc[i, 'AK_RDR'] = str(id[(index+len(pattern)):(index+len(pattern)+10)])

#add program name (lambda method)
df2['Program'] = df2['AK_RDR'].apply(lambda x: 'Seaduck JV' if 'jv' in x else ('Waterfowl' if 'wa' in x else ('Seabrids' if 'sb' in x else ('Landbirds' if 'lb' in x else ('Shorebirds' if 'sh' in x else ('Raptors' if 'ra' in x else ('Science Support' if 'ss' in x else '')))))))

#add operational/exterimental/inactive status
df2['ProjectStatus'] = df2['status'].apply(lambda x: 'Operational' if 'onGoing' in x else ('Inactive' if 'completed' in x else ('inactive' if 'historicalArchive' in x else 'Experimental')))

#output csv
output = df2.to_csv(outCSV, header=df2.columns, index=None, sep =',')